In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import lib.cell as cell
import lib.spec as spec
import numpy as np


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [6]:

cf = spec.Spec.get_configuration_space().sample_configuration()
s = spec.Spec(cf)
while s.valid_spec == False:
    cf = spec.Spec.get_configuration_space().sample_configuration()
    s = spec.Spec(cf)
print(s.matrix)
print(s.ops)

[[0 1 1 1 0 0]
 [0 0 0 1 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 0]]
['input', 'maxpool3x3', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'conv3x3-bn-relu', 'output']


In [7]:
i = tf.keras.Input(shape=(32, 32, 3))
c = cell.Cell(s, 128, i).build()
c = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same')(c)
c = cell.Cell(s, 256, c).build()
c = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same')(c)
c = cell.Cell(s, 512, c).build()
f = tf.reduce_mean(c, [1,2])
o = tf.keras.layers.Dense(10)(f)
model = tf.keras.Model(i, o, name="sample_model")
model.summary()


                   
                                                                 re_lu_38[0][0]                   
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 16, 16, 128)  32768       concatenate_4[0][0]              
__________________________________________________________________________________________________
batch_normalization_40 (BatchNo (None, 16, 16, 128)  512         conv2d_40[0][0]                  
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 16, 16, 128)  32768       concatenate_4[0][0]              
__________________________________________________________________________________________________
re_lu_40 (ReLU)                 (None, 16, 16, 128)  0           batch_normalization_40[0][0]     
_________________________________________________________________________________________

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=4, validation_data=(test_images, test_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 80s 2ms/sample - loss: 1.5496 - accuracy: 0.4235 - val_loss: 4.0239 - val_accuracy: 0.2930
Epoch 2/5
50000/50000 [==============================] - 75s 2ms/sample - loss: 1.1044 - accuracy: 0.6022 - val_loss: 3.5279 - val_accuracy: 0.3153
Epoch 3/5
50000/50000 [==============================] - 75s 1ms/sample - loss: 0.8575 - accuracy: 0.6973 - val_loss: 1.4060 - val_accuracy: 0.5956
Epoch 4/5
50000/50000 [==============================] - 75s 2ms/sample - loss: 0.7037 - accuracy: 0.7562 - val_loss: 1.0638 - val_accuracy: 0.6700
Epoch 5/5
50000/50000 [==============================] - 75s 1ms/sample - loss: 0.5991 - accuracy: 0.7941 - val_loss: 1.0270 - val_accuracy: 0.6847
